In [ ]:
import fitz  # PyMuPDF
import nltk
import networkx as nx  # Add this import
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
from nltk.tokenize import sent_tokenize
from operator import itemgetter
import sklearn

# Getting nltk
import nltk

# Importing word_tokenize to tokenize words in a sentence
from nltk.tokenize import word_tokenize

# Getting stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


stop_words = []
stop_words = set(stop_words)
# stopwords.words('english')
nltk_stopwords = set(stopwords.words('english'))

sklearn_stopwords = set(ENGLISH_STOP_WORDS)
# sklearn_stopwords

total_stopwords = nltk_stopwords.union(sklearn_stopwords)
final_stopwords = total_stopwords.union(stop_words)


def extract_text_from_pdf(pdf_path):
    text = ''
    with fitz.open(pdf_path) as pdf_document:
        num_pages = pdf_document.page_count
        for page_num in range(num_pages):
            page = pdf_document[page_num]
            text += page.get_text()
    return text


def sentence_similarity(sent1, sent2, stopwords=final_stopwords):
    if stopwords is None:
        stopwords = []
    
    sent1 = [word.lower() for word in sent1]
    sent2 = [word.lower() for word in sent2]
    
    all_words = list(set(sent1 + sent2))
    
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    for word in sent1:
        if word not in stopwords:
            vector1[all_words.index(word)] += 1
    
    for word in sent2:
        if word not in stopwords:
            vector2[all_words.index(word)] += 1
    
    return 1 - cosine_distance(vector1, vector2)


def generate_summary(text, max_words=500):
    sentences = sent_tokenize(text)
    stop_words = stopwords.words('english')
    
    sentence_similarity_matrix = np.zeros((len(sentences), len(sentences)))
    
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sentence_similarity_matrix[i][j] = sentence_similarity(
                    sentences[i].split(), sentences[j].split(), stop_words)
    
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    summary = ''
    word_count = 0
    for i in range(len(ranked_sentences)):
        summary += ranked_sentences[i][1] + ' '
        word_count += len(ranked_sentences[i][1].split())
        if word_count >= max_words:
            break
    
    return summary

In [ ]:
# Specify the path to your PDF document
pdf_file_path = '/Users/mukulhooda/Downloads/Operations Management.pdf'

In [ ]:
# Extract text from the PDF document
document_text = extract_text_from_pdf(pdf_file_path)

# Generate a summary under 500 words
document_summary_1 = generate_summary(document_text, max_words=500)

In [ ]:
# Print the summarized text
print(document_summary_1)

In [ ]:

import PyPDF2
import nltk
import networkx as nx
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random  # Import the random module

# Set a seed for the random number generator
random.seed(42)


# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    text = ''
    with open(pdf_file, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)  # Use PdfReader instead of PdfFileReader
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            text += pdf_reader.pages[page_num].extract_text()
    return text

# Function to perform extractive summarization using TextRank
def summarize_text(text, num_sentences=3):
    sentences = sent_tokenize(text)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Create a graph representation of sentences
    graph = nx.from_numpy_array(cosine_matrix)

    # Calculate sentence importance scores using PageRank (TextRank)
    scores = nx.pagerank(graph)

    # Sort sentences by importance score and select the top ones
    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)
    summary = ' '.join([sentence for _, sentence in ranked_sentences[:num_sentences]])

    return summary

# Main function
def main():
    pdf_file = '/Users/mukulhooda/Downloads/Operations Management.pdf'
    text = extract_text_from_pdf(pdf_file)
    summary = summarize_text(text, num_sentences=100)
    print("Summary:")
    print(summary)

if __name__ == "__main__":
    main()


In [ ]:
import fitz  # PyMuPDF
import nltk
import networkx as nx
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
from nltk.tokenize import sent_tokenize
from operator import itemgetter
import sklearn
import re  # Add this import for regular expressions

# Getting nltk
import nltk

# Importing word_tokenize to tokenize words in a sentence
from nltk.tokenize import word_tokenize

# Getting stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

stop_words = []
stop_words = set(stop_words)
nltk_stopwords = set(stopwords.words('english'))

sklearn_stopwords = set(ENGLISH_STOP_WORDS)

total_stopwords = nltk_stopwords.union(sklearn_stopwords)
final_stopwords = total_stopwords.union(stop_words)

# Define a regular expression pattern to identify headings and subheadings
heading_pattern = r'^(#+)\s+(.*?)$'

def extract_headings_and_text(text):
    headings_and_text = []
    lines = text.split('\n')
    current_heading = None
    current_text = []

    for line in lines:
        match = re.match(heading_pattern, line)
        if match:
            # If a heading is found, save the previous heading and text (if any)
            if current_heading and current_text:
                headings_and_text.append((current_heading, '\n'.join(current_text)))
            # Set the current heading and clear the text
            current_heading = match.group(1)
            current_text = []
        else:
            # If not a heading, append the line to the current text
            current_text.append(line)

    # Add the last heading and text (if any)
    if current_heading and current_text:
        headings_and_text.append((current_heading, '\n'.join(current_text)))

    return headings_and_text

def sentence_similarity(sent1, sent2, stopwords=final_stopwords):
    if stopwords is None:
        stopwords = []

    sent1 = [word.lower() for word in sent1]
    sent2 = [word.lower() for word in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for word in sent1:
        if word not in stopwords:
            vector1[all_words.index(word)] += 1

    for word in sent2:
        if word not in stopwords:
            vector2[all_words.index(word)] += 1

    return 1 - cosine_distance(vector1, vector2)

def generate_summary(text, max_words=500):
    headings_and_text = extract_headings_and_text(text)
    stop_words = stopwords.words('english')

    sentence_similarity_matrix = np.zeros((len(headings_and_text), len(headings_and_text)))

    for i in range(len(headings_and_text)):
        for j in range(len(headings_and_text)):
            if i != j:
                sentence_similarity_matrix[i][j] = sentence_similarity(
                    word_tokenize(headings_and_text[i][1]),
                    word_tokenize(headings_and_text[j][1]),
                    stop_words)

    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    ranked_headings_and_text = sorted(((scores[i], h, t) for i, (h, t) in enumerate(headings_and_text)),
                                      key=lambda x: x[0], reverse=True)

    summary = ''
    word_count = 0
    for i in range(len(ranked_headings_and_text)):
        summary += ranked_headings_and_text[i][1] + '\n' + ranked_headings_and_text[i][2] + '\n\n'
        word_count += len(word_tokenize(ranked_headings_and_text[i][2]))
        if word_count >= max_words:
            break

    return summary

In [ ]:
import fitz  # PyMuPDF
import nltk
import networkx as nx
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from operator import itemgetter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Getting nltk
import nltk

# Importing word_tokenize to tokenize words in a sentence
from nltk.tokenize import word_tokenize

# Getting stopwords
from nltk.corpus import stopwords

stop_words = []

# Combine NLTK and sklearn stopwords
nltk_stopwords = set(stopwords.words('english'))
sklearn_stopwords = set(ENGLISH_STOP_WORDS)
total_stopwords = nltk_stopwords.union(sklearn_stopwords)
final_stopwords = total_stopwords.union(stop_words)


def extract_text_from_pdf(pdf_path):
    text = ''
    with fitz.open(pdf_path) as pdf_document:
        num_pages = pdf_document.page_count
        for page_num in range(num_pages):
            page = pdf_document[page_num]
            text += page.get_text()
    return text


def sentence_similarity(sent1, sent2, stopwords=final_stopwords):
    if stopwords is None:
        stopwords = []

    sent1 = [word.lower() for word in sent1]
    sent2 = [word.lower() for word in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for word in sent1:
        if word not in stopwords:
            vector1[all_words.index(word)] += 1

    for word in sent2:
        if word not in stopwords:
            vector2[all_words.index(word)] += 1

    return 1 - cosine_distance(vector1, vector2)


def generate_summary(text, max_words=500):
    sentences = sent_tokenize(text)
    stop_words = stopwords.words('english')

    # Initialize a flag to check if a sentence is a heading or subheading
    is_heading = False
    summary = ''

    for sentence in sentences:
        if sentence.isupper():  # Check if the sentence is in all uppercase (a heading)
            is_heading = True
            summary += sentence + '\n'  # Add a line break after the heading
        elif sentence.strip() == '':  # Check if the sentence is empty (end of a paragraph)
            is_heading = False
        elif is_heading:
            summary += sentence + '\n'  # Add a line break after a subheading
        else:
            summary += sentence + ' '

    return summary


# Example usage:
pdf_path = pdf_file_path
text = extract_text_from_pdf(pdf_path)
summary = generate_summary(text)
print(summary)


In [ ]:
import fitz  # PyMuPDF
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re

# Getting stopwords
from nltk.corpus import stopwords

stop_words = []

# Combine NLTK and sklearn stopwords
nltk_stopwords = set(stopwords.words('english'))
sklearn_stopwords = set(ENGLISH_STOP_WORDS)
total_stopwords = nltk_stopwords.union(sklearn_stopwords)
final_stopwords = total_stopwords.union(stop_words)


def extract_text_from_pdf(pdf_path):
    text = ''
    with fitz.open(pdf_path) as pdf_document:
        num_pages = pdf_document.page_count
        for page_num in range(num_pages):
            page = pdf_document[page_num]
            text += page.get_text()
    return text


def sentence_similarity(sent1, sent2, stopwords=final_stopwords):
    if stopwords is None:
        stopwords = []

    sent1 = [word.lower() for word in sent1]
    sent2 = [word.lower() for word in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for word in sent1:
        if word not in stopwords:
            vector1[all_words.index(word)] += 1

    for word in sent2:
        if word not in stopwords:
            vector2[all_words.index(word)] += 1

    return 1 - cosine_distance(vector1, vector2)


def generate_summary(text, max_words=500):
    sentences = sent_tokenize(text)
    stop_words = stopwords.words('english')

    # Initialize a flag to check if a sentence is a heading or subheading
    is_heading = False
    summary = ''

    for sentence in sentences:
        # Check if the sentence contains mostly uppercase words (likely a heading or subheading)
        if re.match(r'^[A-Z\s]+$', sentence.strip()):
            is_heading = True
            summary += sentence + '\n'  # Add a line break after the heading
        elif sentence.strip() == '':  # Check if the sentence is empty (end of a paragraph)
            is_heading = False
        elif is_heading:
            summary += sentence + '\n'  # Add a line break after a subheading
        else:
            summary += sentence + ' '

    return summary


# Example usage:
pdf_path = pdf_file_path
text = extract_text_from_pdf(pdf_path)
summary = generate_summary(text)
print(summary)


In [ ]:
text=summary
# Split the text into paragraphs based on double line breaks (empty lines)
paragraphs = text.split('\n\n')

# Create a PDF document
pdf_document = fitz.open()

# Loop through paragraphs and add them as text to the PDF with line breaks
for paragraph in paragraphs:
    pdf_page = pdf_document.new_page()
    pdf_page.insert_text(fitz.Point(50, 50), paragraph, fontname="helvetica")

# Save the PDF to a file
output_pdf_path = 'summary_output.pdf'
pdf_document.save(output_pdf_path)
pdf_document.close()

print("PDF file saved:", output_pdf_path)

# PDF Summarizer 

In [1]:
import fitz  # PyMuPDF
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re

# Getting stopwords
from nltk.corpus import stopwords

stop_words = []

# Combine NLTK and sklearn stopwords
nltk_stopwords = set(stopwords.words('english'))
sklearn_stopwords = set(ENGLISH_STOP_WORDS)
total_stopwords = nltk_stopwords.union(sklearn_stopwords)
final_stopwords = total_stopwords.union(stop_words)


def extract_text_from_pdf(pdf_path):
    text = ''
    with fitz.open(pdf_path) as pdf_document:
        num_pages = pdf_document.page_count
        for page_num in range(num_pages):
            page = pdf_document[page_num]
            text += page.get_text()
    return text


def sentence_similarity(sent1, sent2, stopwords=final_stopwords):
    if stopwords is None:
        stopwords = []

    sent1 = [word.lower() for word in sent1]
    sent2 = [word.lower() for word in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for word in sent1:
        if word not in stopwords:
            vector1[all_words.index(word)] += 1

    for word in sent2:
        if word not in stopwords:
            vector2[all_words.index(word)] += 1

    return 1 - cosine_distance(vector1, vector2)


def generate_summary(text, max_words=500):
    sentences = sent_tokenize(text)
    stop_words = stopwords.words('english')

    # Initialize a flag to check if a sentence is a heading or subheading
    is_heading = False
    summary = ''
    word_count = 0

    for sentence in sentences:
        # Check if the sentence contains mostly uppercase words (likely a heading or subheading)
        if re.match(r'^[A-Z\s]+$', sentence.strip()):
            is_heading = True
            summary += sentence + '\n'  # Add a line break after the heading
        elif sentence.strip() == '':  # Check if the sentence is empty (end of a paragraph)
            is_heading = False
        elif is_heading:
            summary += sentence + '\n'  # Add a line break after a subheading
        else:
            # Calculate the word count in the current sentence
            word_count += len(sentence.split())
            # If adding the sentence doesn't exceed the word limit, add it to the summary
            if word_count <= max_words:
                summary += sentence + ' '
            else:
                break  # Stop summarizing when the word limit is reached

    return summary


pdf_path = input('Enter your pdf file: ')
text = extract_text_from_pdf(pdf_path)
summary = generate_summary(text, max_words=500)
print(summary)


Enter your pdf file: /Users/mukulhooda/Desktop/Projects/My Projects /Project --> PDF Summarizer /Operations Management.pdf
Operations Management: Oil and Gas
Report
Introduction
Operations management is a branch of management that deals with the
designing and supervision of operational processes in a business organization. Operations management covers the responsibility over all processes that involve
the production of goods and services as well as the delivery of such productions
to the final consumers. In its duties, an operations management department ensures that processes are
planned for and executed in an efficient and effective way to satisfy the needs of
the organization and its customers. This paper seeks to discuss concepts of
operations management. The paper will look into the history, functions, case studies, advantages,
disadvantages and factors that affect the department among others. The paper
will then look into the operations management’s involvement in oil and gas
com

In [2]:
text=summary
# Split the text into paragraphs based on double line breaks (empty lines)
paragraphs = text.split('\n\n')

# Create a PDF document
pdf_document = fitz.open()

# Loop through paragraphs and add them as text to the PDF with line breaks
for paragraph in paragraphs:
    pdf_page = pdf_document.new_page()
    pdf_page.insert_text(fitz.Point(50, 50), paragraph, fontname="helvetica")

# Save the PDF to a file
output_pdf_path = 'summary_output.pdf'
pdf_document.save(output_pdf_path)
pdf_document.close()

print("PDF file saved:", output_pdf_path)

PDF file saved: summary_output.pdf
